# Setup

In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# from econml.dml import CausalForestDML

In [2]:
df = pd.read_csv('pp_data_preop_6month.csv')

In [3]:
data = df[df['SRS-22\\21. (SRS)@6-month-PO'] >= 3.4]

In [4]:
data

,PID,Race_White,Race_Black,Race_Other,Race_Hispanic,Race_Asian,Gender,Age,Comorbidity,Lateral Radiographs\Lateral C7 to Sacrum@Pre-OP,...,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T5,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T6,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T7,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T8,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T9,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_C7,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T1,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T2,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T3,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T4
0,GRNDRUY0001,-1.670172,-0.502169,-0.137361,5.916080,-0.128338,0.509387,-0.468418,1.721425,0.822326,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,GRNDRUY0002,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,0.487986,1.721425,-0.949772,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,GRNDRUY0004,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,-0.468418,-0.580914,1.030808,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,GRNDRUY0005,-1.670172,1.991361,-0.137361,-0.169031,-0.128338,0.509387,0.487986,1.721425,-0.810784,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,GRNDRUY0013,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,-1.963144,0.966188,-0.580914,1.378279,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,GRNDRUY1022,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,0.487986,-0.580914,0.301121,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
365,GRNDRUY1037,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,0.487986,-0.580914,-1.193001,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
366,GRNDRUY1042,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,0.679267,1.721425,-0.393820,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
367,GRNDRUY1045,0.598741,-0.502169,-0.137361,-0.169031,-0.128338,0.509387,0.487986,-0.580914,-0.081096,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Preprocessing

In [5]:
target = data[['SRS-22\\21. (SRS)@6-month-PO']]

In [6]:
treatment_df = data.filter(like='@6-month')
control_df = data.filter(like='@Pre-OP')

In [7]:
control_df

,Lateral Radiographs\Lateral C7 to Sacrum@Pre-OP,Lateral Radiographs\Lordosis (Top of 12 - Top of Sacrum)@Pre-OP,Lateral Radiographs\T10 - L2@Pre-OP,Lateral Radiographs\T2-T12@Pre-OP,Lateral Radiographs\T2-T5@Pre-OP,Lateral Radiographs\T5 - T12@Pre-OP,Posterior/Anterior Radiographs\Coronal C7 to CSVL@Pre-OP,Posterior/Anterior Radiographs\Lumbar Bend:@Pre-OP,Posterior/Anterior Radiographs\Lumbar Curve Direction:@Pre-OP,Posterior/Anterior Radiographs\Lumbar Curve:@Pre-OP,...,Radiographic Information\Stable Vertebrae@Pre-OP_L1,Radiographic Information\Stable Vertebrae@Pre-OP_L2,Radiographic Information\Stable Vertebrae@Pre-OP_L3,Radiographic Information\Stable Vertebrae@Pre-OP_L4,Radiographic Information\Stable Vertebrae@Pre-OP_L5,Radiographic Information\Stable Vertebrae@Pre-OP_T10,Radiographic Information\Stable Vertebrae@Pre-OP_T11,Radiographic Information\Stable Vertebrae@Pre-OP_T12,Radiographic Information\Stable Vertebrae@Pre-OP_T8,Radiographic Information\Stable Vertebrae@Pre-OP_T9
0,0.822326,-0.980141,0.024899,-2.359679,-0.470566,-1.492188,0.316739,1.923216,-0.259808,1.011582,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.949772,-0.378940,0.085830,0.611242,-1.211559,0.823832,0.759459,-0.170013,-0.259808,-0.441244,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.030808,-0.679541,-0.139842,0.391119,2.359004,-1.427019,0.290901,-0.065752,3.849002,-1.838193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.810784,1.828097,-1.006419,-1.054352,-1.320983,-0.221743,-0.944840,-0.298333,-0.259808,-1.105571,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.378279,-2.230004,3.775556,-1.611996,-1.028392,-1.240104,-1.625566,1.289634,3.849002,1.023999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,0.301121,0.554502,-0.557333,-0.078476,0.693851,-0.300519,0.109539,-0.131998,-0.259808,-1.161449,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
365,-1.193001,-1.359846,-0.455781,-0.335285,0.729533,-0.458072,1.097337,2.765320,-0.259808,2.451991,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
366,-0.393820,1.242718,0.255083,0.618579,-0.719150,1.088806,1.037712,-0.629561,-0.259808,-0.081142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
367,-0.081096,-0.133714,1.417291,-1.810840,-0.547877,-1.558074,0.729646,-0.485200,-0.259808,-0.478496,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Selecting the right Rediographys

In [8]:
columns_to_remove = [
    'SRS Normalized Scores\\General Function (SRS 22)@6-month-PO',
    'SRS Normalized Scores\\Mental Health (SRS 22)@6-month-PO',
    'SRS Normalized Scores\\Pain (SRS 22)@6-month-PO',
    'SRS Normalized Scores\\Satisfaction (SRS 22)@6-month-PO',
    'SRS Normalized Scores\\Self-Image (SRS 22)@6-month-PO',
    'SRS Normalized Scores\\Total (SRS 22)@6-month-PO',
    'SRS-22\\1. (SRS)@6-month-PO',
    'SRS-22\\10. (SRS)@6-month-PO',
    'SRS-22\\11. (SRS)@6-month-PO',
    'SRS-22\\12. (SRS)@6-month-PO',
    'SRS-22\\13. (SRS)@6-month-PO',
    'SRS-22\\14. (SRS)@6-month-PO',
    'SRS-22\\15. (SRS)@6-month-PO',
    'SRS-22\\16. (SRS)@6-month-PO',
    'SRS-22\\17. (SRS)@6-month-PO',
    'SRS-22\\18. (SRS)@6-month-PO',
    'SRS-22\\19. (SRS)@6-month-PO',
    'SRS-22\\2. (SRS)@6-month-PO',
    'SRS-22\\20. (SRS)@6-month-PO',
    'SRS-22\\22. (SRS)@6-month-PO',
    'SRS-22\\3. (SRS)@6-month-PO',
    'SRS-22\\4. (SRS)@6-month-PO',
    'SRS-22\\5. (SRS)@6-month-PO',
    'SRS-22\\6. (SRS)@6-month-PO',
    'SRS-22\\7. (SRS)@6-month-PO',
    'SRS-22\\8. (SRS)@6-month-PO',
    'SRS-22\\9. (SRS)@6-month-PO'
]

columns_to_r = ['SRS Normalized Scores\\General Function (SRS 22)@Pre-OP',
 'SRS Normalized Scores\\Mental Health (SRS 22)@Pre-OP',
 'SRS Normalized Scores\\Pain (SRS 22)@Pre-OP',
 'SRS Normalized Scores\\Satisfaction (SRS 22)@Pre-OP',
 'SRS Normalized Scores\\Self-Image (SRS 22)@Pre-OP',
 'SRS Normalized Scores\\Total (SRS 22)@Pre-OP',
 'SRS-22\\1. (SRS)@Pre-OP',
 'SRS-22\\10. (SRS)@Pre-OP',
 'SRS-22\\11. (SRS)@Pre-OP',
 'SRS-22\\12. (SRS)@Pre-OP',
 'SRS-22\\13. (SRS)@Pre-OP',
 'SRS-22\\14. (SRS)@Pre-OP',
 'SRS-22\\15. (SRS)@Pre-OP',
 'SRS-22\\16. (SRS)@Pre-OP',
 'SRS-22\\17. (SRS)@Pre-OP',
 'SRS-22\\18. (SRS)@Pre-OP',
 'SRS-22\\19. (SRS)@Pre-OP',
 'SRS-22\\2. (SRS)@Pre-OP',
 'SRS-22\\20. (SRS)@Pre-OP',
 'SRS-22\\21. (SRS)@Pre-OP',
 'SRS-22\\22. (SRS)@Pre-OP',
 'SRS-22\\3. (SRS)@Pre-OP',
 'SRS-22\\4. (SRS)@Pre-OP',
 'SRS-22\\5. (SRS)@Pre-OP',
 'SRS-22\\6. (SRS)@Pre-OP',
 'SRS-22\\7. (SRS)@Pre-OP',
 'SRS-22\\8. (SRS)@Pre-OP',
 'SRS-22\\9. (SRS)@Pre-OP']
treatment_df_ = treatment_df.drop(columns=columns_to_remove)
control_df_ = control_df.drop(columns=columns_to_r)

In [9]:
treatment_df_

,Lateral Radiographs\Junctional Kyphosis - Distal@6-month-PO,Lateral Radiographs\Junctional Kyphosis - Proximal@6-month-PO,Lateral Radiographs\Lateral C7 to Sacrum@6-month-PO,Lateral Radiographs\Lordosis (Top of 12 - Top of Sacrum)@6-month-PO,Lateral Radiographs\T10 - L2@6-month-PO,Lateral Radiographs\T2-T12@6-month-PO,Lateral Radiographs\T2-T5@6-month-PO,Lateral Radiographs\T5 - T12@6-month-PO,Posterior/Anterior Radiographs\Coronal C7 to CSVL@6-month-PO,Posterior/Anterior Radiographs\Disc Angulation - Below EIV@6-month-PO,...,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T5,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T6,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T7,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T8,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Lower:@6-month-PO_T9,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_C7,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T1,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T2,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T3,Posterior/Anterior Radiographs\Upper Thoracic Levels Measured Upper:@6-month-PO_T4
0,0.476206,1.299583,-0.346977,0.438259,1.627740,0.168015,1.555762,-1.314482,1.202001,0.426114,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.426168,-0.866455,-0.611229,-0.076124,0.296996,-0.994037,-1.337416,-0.453419,1.727366,0.601434,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.366079,-0.810602,-0.950981,-0.739604,1.285647,-1.106494,-1.248271,-0.576869,0.454885,0.178248,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-1.005303,-0.580210,1.012031,-0.419046,-1.249268,0.036816,-0.819909,0.359294,-0.416579,-0.677191,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.527528,0.069078,0.068276,0.244434,-0.925989,1.405039,-1.533073,1.562931,-0.697395,-0.922034,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,-0.687245,-0.833292,-0.082725,1.429751,-0.272590,0.196129,-0.192417,0.462169,-0.019706,-0.958307,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
365,0.020357,-1.021795,-1.592734,-0.553233,-0.922568,0.336700,0.235945,0.585619,0.959525,1.568715,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
366,-1.012556,0.292489,-2.891342,0.989917,-1.083352,-0.169355,-0.986623,0.750218,-1.640358,-0.121005,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
367,-0.398195,-0.348072,0.370277,-0.061214,2.004044,-1.434492,-1.531916,-1.534635,2.338738,0.102679,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
control_df_

,Lateral Radiographs\Lateral C7 to Sacrum@Pre-OP,Lateral Radiographs\Lordosis (Top of 12 - Top of Sacrum)@Pre-OP,Lateral Radiographs\T10 - L2@Pre-OP,Lateral Radiographs\T2-T12@Pre-OP,Lateral Radiographs\T2-T5@Pre-OP,Lateral Radiographs\T5 - T12@Pre-OP,Posterior/Anterior Radiographs\Coronal C7 to CSVL@Pre-OP,Posterior/Anterior Radiographs\Lumbar Bend:@Pre-OP,Posterior/Anterior Radiographs\Lumbar Curve Direction:@Pre-OP,Posterior/Anterior Radiographs\Lumbar Curve:@Pre-OP,...,Radiographic Information\Stable Vertebrae@Pre-OP_L1,Radiographic Information\Stable Vertebrae@Pre-OP_L2,Radiographic Information\Stable Vertebrae@Pre-OP_L3,Radiographic Information\Stable Vertebrae@Pre-OP_L4,Radiographic Information\Stable Vertebrae@Pre-OP_L5,Radiographic Information\Stable Vertebrae@Pre-OP_T10,Radiographic Information\Stable Vertebrae@Pre-OP_T11,Radiographic Information\Stable Vertebrae@Pre-OP_T12,Radiographic Information\Stable Vertebrae@Pre-OP_T8,Radiographic Information\Stable Vertebrae@Pre-OP_T9
0,0.822326,-0.980141,0.024899,-2.359679,-0.470566,-1.492188,0.316739,1.923216,-0.259808,1.011582,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.949772,-0.378940,0.085830,0.611242,-1.211559,0.823832,0.759459,-0.170013,-0.259808,-0.441244,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.030808,-0.679541,-0.139842,0.391119,2.359004,-1.427019,0.290901,-0.065752,3.849002,-1.838193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.810784,1.828097,-1.006419,-1.054352,-1.320983,-0.221743,-0.944840,-0.298333,-0.259808,-1.105571,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.378279,-2.230004,3.775556,-1.611996,-1.028392,-1.240104,-1.625566,1.289634,3.849002,1.023999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,0.301121,0.554502,-0.557333,-0.078476,0.693851,-0.300519,0.109539,-0.131998,-0.259808,-1.161449,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
365,-1.193001,-1.359846,-0.455781,-0.335285,0.729533,-0.458072,1.097337,2.765320,-0.259808,2.451991,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
366,-0.393820,1.242718,0.255083,0.618579,-0.719150,1.088806,1.037712,-0.629561,-0.259808,-0.081142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
367,-0.081096,-0.133714,1.417291,-1.810840,-0.547877,-1.558074,0.729646,-0.485200,-0.259808,-0.478496,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Correlation to make sure same features are in both cases

In [11]:
# Function to normalize column names
def normalize_columns(df):
    df.columns = df.columns.str.replace(r'@Pre-OP', '', regex=True)
    df.columns = df.columns.str.replace(r'@6-month-PO', '', regex=True)
    return df


In [12]:
df1_normalized = normalize_columns(control_df_.copy())
df2_normalized = normalize_columns(treatment_df_.copy())


# Create a mapping from normalized names to original names
df1_column_mapping = dict(zip(df1_normalized, control_df_.columns))
df2_column_mapping = dict(zip(df2_normalized, treatment_df_.columns))

In [13]:
# Find common columns
common_columns = df1_normalized.columns.intersection(df2_normalized.columns)

In [14]:
# Map back to the original columns
common_columns_df1 = [df1_column_mapping[col] for col in common_columns]
common_columns_df2 = [df2_column_mapping[col] for col in common_columns]

In [15]:
# Filter both DataFrames to keep only common columns
df1_filtered = control_df_[common_columns_df1]
df2_filtered = treatment_df_[common_columns_df2]

# Demographics info removed

In [16]:
# List of columns to select
columns_to_select = [
    'Race_White', 'Race_Black', 'Race_Other', 'Race_Hispanic',
    'Race_Asian', 'Gender', 'Age', 'Comorbidity'
]

dem = data[columns_to_select]

In [17]:
# Merge the DataFrames along columns
control = pd.concat([dem, df1_filtered ], axis=1)
treatment = pd.concat([dem, df2_filtered], axis=1)

# Train/Test

In [62]:
# Combine DataFrames
# Split the combined DataFrame
train_combined, test_combined = train_test_split(data, test_size=0.3, random_state=35)

In [63]:
y_train = train_combined[target.columns]

# Smote

In [64]:
# Remove the 'ID' column
train_combined = train_combined.drop(columns=['PID'])

In [65]:
# Apply SMOTE to balance the classes
smote = SMOTE(random_state=35)
X_train_smote, y_train_smote = smote.fit_resample(train_combined, np.array(y_train.astype(int)).ravel())

In [66]:
# Separate the combined DataFrames back into original forms
control_train = X_train_smote[control.columns]
treatment_train = X_train_smote[treatment.columns]


control_test = test_combined[control.columns]
treatment_test = test_combined[treatment.columns]
y_test = test_combined[target.columns]

In [67]:
# Map target values to binary [0, 1] after splitting
# y_train = y_train['SRS-22\\21. (SRS)@6-month-PO'].map({4: 0, 5: 1})
y_train_smote = pd.Series(y_train_smote)
y_train_smote = y_train_smote.map({4: 0, 5: 1})
y_test = pd.Series(np.array(y_test.astype(int)).ravel())

y_test = y_test.map({4: 0, 5: 1})


# Modeling

In [68]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

# Control Group

# CatBoost

In [81]:
import joblib


# Load the model from the file
catboost_clf = joblib.load('control_cat_boosting_model.joblib')

     iterations  test-AUC-mean  test-AUC-std  test-Logloss-mean  \
0             0       0.722645      0.047698           0.692843   
1             1       0.744875      0.088452           0.692629   
2             2       0.761704      0.053302           0.692382   
3             3       0.798961      0.064319           0.692077   
4             4       0.819044      0.067299           0.691777   
..          ...            ...           ...                ...   
995         995       0.928947      0.032136           0.522870   
996         996       0.929224      0.031808           0.522718   
997         997       0.929363      0.031648           0.522610   
998         998       0.929224      0.031536           0.522470   
999         999       0.929363      0.031648           0.522374   

     test-Logloss-std  train-Logloss-mean  train-Logloss-std  
0            0.000093            0.692602           0.000093  
1            0.000162            0.692147           0.000104  
2     

In [82]:
y_pred = catboost_clf.predict(control_test)

In [83]:
accuracy = accuracy_score(np.array(y_test), y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.74


In [85]:
from sklearn.metrics import roc_auc_score
y_pred_prob = catboost_clf.predict_proba(control_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)
print(auc)

0.6921052631578947


# Gradient Boosting

In [86]:
# Load the model from the file
best_model_gb = joblib.load('control_best_gradient_boosting_model.joblib')

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [88]:

y_pred = best_model_gb.predict(control_test)

In [89]:
accuracy = accuracy_score(np.array(y_test), y_pred)

print(f"Accuracy: {accuracy:.2f}")
y_pred_prob = best_model_gb.predict_proba(control_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)
print(auc)

Accuracy: 0.75
0.6436090225563911


# Treatment Group

# CatBoost

In [93]:
# Load the model from the file
catboost_clf = joblib.load('treatment_cat_boosting_model.joblib')

     iterations  test-AUC-mean  test-AUC-std  test-Logloss-mean  \
0             0       0.664751      0.036976           0.692932   
1             1       0.749861      0.052545           0.692627   
2             2       0.778255      0.035384           0.692424   
3             3       0.789197      0.037882           0.692218   
4             4       0.818075      0.030647           0.691919   
..          ...            ...           ...                ...   
995         995       0.926593      0.009976           0.531709   
996         996       0.926454      0.009860           0.531590   
997         997       0.926454      0.009860           0.531522   
998         998       0.926454      0.009860           0.531423   
999         999       0.926316      0.010126           0.531329   

     test-Logloss-std  train-Logloss-mean  train-Logloss-std  
0            0.000061            0.692693           0.000033  
1            0.000066            0.692243           0.000122  
2     

In [94]:
y_pred = catboost_clf.predict(treatment_test)

In [95]:
accuracy = accuracy_score(np.array(y_test), y_pred)

print(f"Accuracy: {accuracy:.2f}")
y_pred_prob = catboost_clf.predict_proba(treatment_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)
print(auc)


Accuracy: 0.66
0.5887218045112781


# Gradient Boosting

In [97]:
best_model_gb = joblib.load('treatment_best_gradient_boosting_model.joblib')

GradientBoostingClassifier(learning_rate=0.05, max_depth=5, n_estimators=200)

In [100]:
best_model_gb.fit(treatment_train, y_train_smote)
y_pred = best_model_gb.predict(treatment_test)

In [101]:
accuracy = accuracy_score(np.array(y_test), y_pred)

print(f"Accuracy: {accuracy:.2f}")
y_pred_prob = best_model_gb.predict_proba(treatment_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)
print(auc)

Accuracy: 0.69
0.6225563909774436
